In [1]:
from data_algebra.data_ops import *
import data_algebra.env

_, _1, _2, _get = [None, None, None, lambda x: x] # don't look unbound
data_algebra.env.push_onto_namespace_stack(locals())



t1 = TableDescription('t1', ['x', 'y'])
t2 = TableDescription('t2', ['x', 'z'])

ops = t1 >> Extend({'x':'x + 1'}) >> NaturalJoin(b=t2, by='x')
print(ops)



t1 >>
    Extend({'x': (x + 1)}) >>
    NaturalJoin(b=t2, by=x, jointype=INNER)
